<a href="https://colab.research.google.com/github/Nikhilam07/PRODIGY_ML_03/blob/main/ML_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
# 📦 STEP 2: Unzip to /content/image.zip
!unzip image.zip -d /content


Archive:  image.zip
replace /content/image/train/cats/cat.0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Import Libraries

In [49]:
# 📁 STEP 5: Load Dataset
train_dir = '/content/image/train'
val_dir = '/content/image/validation'

X_train, y_train = load_images_from_folder(train_dir)
X_val, y_val = load_images_from_folder(val_dir)


Loading dogs: 100%|██████████| 500/500 [00:00<00:00, 992.19it/s] 


In [50]:
pip install opencv-python scikit-image scikit-learn tqdm


In [51]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score


In [52]:
def extract_hog_features(folder_path, label_map, image_size=(128, 128)):
    features, labels = [], []
    for class_name in ['cats', 'dogs']:
        path = os.path.join(folder_path, class_name)
        for filename in tqdm(os.listdir(path), desc=f"Loading {class_name}"):
            img_path = os.path.join(path, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, image_size)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            hog_feat = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                           cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
            features.append(hog_feat)
            labels.append(label_map[class_name])
    return np.array(features), np.array(labels)

label_map = {'cats': 0, 'dogs': 1}
train_dir = 'image/train'
val_dir = 'image/validation'

X_train, y_train = extract_hog_features(train_dir, label_map)
X_val, y_val = extract_hog_features(val_dir, label_map)


Loading dogs: 100%|██████████| 500/500 [00:04<00:00, 122.29it/s]


In [53]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

svm = SVC(kernel='rbf', C=10, gamma='scale', class_weight='balanced')
svm.fit(X_train_scaled, y_train)


SVC(C=10, class_weight='balanced')

In [54]:
y_pred = svm.predict(X_val_scaled)

print("📈 Classification Report:\n")
print(classification_report(y_val, y_pred, target_names=['Cat', 'Dog']))
print(f"✅ Accuracy: {accuracy_score(y_val, y_pred) * 100:.2f}%")


📈 Classification Report:

              precision    recall  f1-score   support

         Cat       0.71      0.74      0.72       500
         Dog       0.73      0.70      0.72       500

    accuracy                           0.72      1000
   macro avg       0.72      0.72      0.72      1000
weighted avg       0.72      0.72      0.72      1000

✅ Accuracy: 72.00%


In [58]:
# 🧪 STEP 9: Predict New Image (Optional)
def predict_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image from {image_path}")
        return None

    # Resize and convert to grayscale as done during training for HOG
    img = cv2.resize(img, (128, 128))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Extract HOG features
    hog_feat = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)

    # Reshape for the scaler
    hog_feat = hog_feat.reshape(1, -1)

    # Scale the HOG features
    scaled = scaler.transform(hog_feat)

    # Predict
    pred = svm.predict(scaled)[0]
    return "Dog" if pred == 1 else "Cat"

# Test Example:
# Make sure to use a valid image path from your dataset
print(predict_image('/content/image/Säugende_Hündin.JPG'))  # or any file path

Dog
